In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage') 
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text

print("목표 키워드 :",target_keyword)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN)  #RETURN = enter
time.sleep(5)

print("검색 완료. 현재 페이지 타이틀 :",driver.title)
driver.quit()

목표 키워드 : 반팔
검색 완료. 현재 페이지 타이틀 : 반팔 | 무신사 추천 브랜드


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re # remove의 약자. 무언가(요소)를 제거할 때 필요한 라이브러리 객체

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage') 
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
# print("목표 키워드 :",target_keyword)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN)  #RETURN = enter
time.sleep(5)



scroll_pause = 2

for _ in range(10) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)

products = driver.find_elements(By.CSS_SELECTOR, "div.sc-d36st-1.elXjMR")[:100]
product_data = []

for product in products :
    try :
        name = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip()
        url = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href")
        review_info = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text
        review_count = int(re.sub(r"[^\d]", "", review_info)) if review_info else 0 #삼항조건연산자 -> 조건문을 한 줄로 간단히 표현하는 문법
        product_data.append({"상품명": name, "URL" : url, "리뷰수" : review_count})
       
    except :
        continue  #에러가 난다면 이 단계를 뛰어넘게 하는 명령어

df = pd.DataFrame(product_data)
df_top30 = df.sort_values(by="리뷰수", ascending=False).head(30).reset_index(drop=True)

driver.quit()
df_top30.head()

,상품명,URL,리뷰수
0,Rowdy Basketball League T-Shirt_Oatmeal,https://www.musinsa.com/products/4147228,724
1,UNIVERSITY PIGMENT TEE CHARCOAL(MG2CMMT531A),https://www.musinsa.com/products/2404968,712
2,원캣 우먼 레귤러 베이직 티셔츠,https://www.musinsa.com/products/3401498,705
3,[2PACK]돌핀 아치로고 티셔츠 화이트 블랙 2팩,https://www.musinsa.com/products/3060324,585
4,[쿨탠다드] LV2 액티브 숏 슬리브 (3color),https://www.musinsa.com/products/3535315,425


In [10]:
from openpyxl import Workbook

driver = webdriver.Chrome(service=service, options=options)
all_reviews = []

for idx, row in df_top30.iterrows() :
    print(f"{idx+1}번째 상품리뷰 수집 중 :", {row['상품명']})
    driver.get(row['URL'])
    time.sleep(2)

    try :
        review_tab = driver.find_element(By.XPATH, "//button[@data-button-id='prd_review_tab']")
        review_tab.click()
        time.sleep(2)

        for _ in range(3) :
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)

        review_elements = driver.find_elements(By.CSS_SELECTOR, ".text-body_13px_reg.text-black.font-pretendard")
        for review in review_elements[:10] :
            text = review.text.strip()
            if text :
                all_reviews.append({
                    "상품명": row["상품명"],
                    "리뷰": text
                })

    except Exception as e :
        print("리뷰 수집 실패:",e)
        continue

driver.quit()

wb = Workbook()
ws = wb.active
ws.title = "무신사 리뷰"

ws.append(["상품명","리뷰"])

for r in all_reviews :
    ws.append([r["상품명"], r["리뷰"]])

wb.save("musinsa_top30_reviews.xlsx")
print("✅엑셀 저장 완료: musinsa_top30_reviews.xlsx")

1번째 상품리뷰 수집 중 : {'Rowdy Basketball League T-Shirt_Oatmeal'}
2번째 상품리뷰 수집 중 : {'UNIVERSITY PIGMENT TEE CHARCOAL(MG2CMMT531A)'}
3번째 상품리뷰 수집 중 : {'원캣 우먼 레귤러 베이직 티셔츠'}
4번째 상품리뷰 수집 중 : {'[2PACK]돌핀 아치로고 티셔츠 화이트 블랙 2팩'}
5번째 상품리뷰 수집 중 : {'[쿨탠다드] LV2 액티브 숏 슬리브 (3color)'}
6번째 상품리뷰 수집 중 : {'NM ENGINE T-SHIRT - BLACK'}
7번째 상품리뷰 수집 중 : {'AECA SOCIAL BALANCE HALF SLEEVE-WASHED CHARCOAL'}
8번째 상품리뷰 수집 중 : {'스탠다드 무지 라운드 반팔 티셔츠 HA-1030 HAR1002 멜란지'}
9번째 상품리뷰 수집 중 : {'빈티지 하트 티셔츠 WHITE-BLUE'}
10번째 상품리뷰 수집 중 : {'WAKE UP 쿨 코튼 오버핏 반팔(SISSTD-0053)'}
11번째 상품리뷰 수집 중 : {'오버핏 크루넥 나인티스팝 로고 반팔 티셔츠 블랙'}
12번째 상품리뷰 수집 중 : {'냥냥냥 티셔츠 [버건디]'}
13번째 상품리뷰 수집 중 : {'Identity Addiction T-Shirt - Gray'}
14번째 상품리뷰 수집 중 : {'CDC T-Rex Heavyweight Tee Vintage Black'}
15번째 상품리뷰 수집 중 : {'MOZZI MEONZI 반팔티 [4컬러]'}
16번째 상품리뷰 수집 중 : {'[20수] DTP HEART Drawing 반팔티 화이트'}
17번째 상품리뷰 수집 중 : {'SWIPE 4 FUTURE TEE (BURGUNDY)'}
18번째 상품리뷰 수집 중 : {'ICE LITE 초냉감 세미오버핏 티셔츠'}
19번째 상품리뷰 수집 중 : {'IWTL 쿨코튼 티셔츠'}
20번째 상품리뷰 수집 중 : {'Speed Track nylon T-shirt

In [11]:
pip install sqlalchemy # 파이썬에서 특정 db를 좀 더 편하게 다룰 수 있게 해주는 라이브러리

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

EXCEL_PATH = "musinsa_top30_reviews.xlsx"
DB_NAME = "musinsa_db"
TABLE_NAME = "reviews"
USER = "root"
PASSWORD = "forena1303!!"
HOST = "127.0.0.1"
PORT = 3306

df = pd.read_excel(EXCEL_PATH)

conn = pymysql.connect(host=HOST, user=USER, password=PASSWORD, port=PORT)
cursor = conn.cursor()
cursor.execute(
    f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;" #utf8mb4 -> 표준문자규약
)
conn.close()

engine_str = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(engine_str)

df.to_sql(name=TABLE_NAME, con=engine, if_exists="replace", index=False)
print(f"✅데이터가 MySQL [{DB_NAME}]의 테이블 [{TABLE_NAME}]에 성공적으로 저장되었습니다.")

✅데이터가 MySQL [musinsa_db]의 테이블 [reviews]에 성공적으로 저장되었습니다.
